# Data Wrangling

In [9]:
import pandas as pd 
import plotly.express as px 
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.ar_model import AutoReg

**Importing Data**

In [11]:
df = pd.read_csv("data/fraud_data.csv")
df.head()

,Year,Month,Prepaid Cards on ATMs,Charge Cards on ATMs,Credit Cards on ATMs,Debit Cards on ATMs,"POS Machines (Debit, Credit, Prepaid, Charge Cards)",Total,Fraud Volumes (In Millions)
0,2024,June,213.0,0.0,639,41914,22308,42766,5760.5802
1,2024,May,302.0,0.0,680,47979,23916,48961,6595.0467
2,2024,April,234.0,0.0,623,45833,23579,46691,6289.2777
3,2024,March,264.0,0.0,619,50850,24034,51733,6968.4351
4,2024,February,264.0,0.0,584,43488,22657,44335,5971.9245
